# 라이브러리 임포트

In [456]:
import pandas as pd
import numpy as np
import openpyxl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm

# 한글폰트 설정
plt.rc('font', family='NanumBarunGothic')

# 데이터 출력 생략 없애기
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [457]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [458]:
data_root = '/root/ML-Regression/Competition/data/'

# 데이터 불러오기

In [459]:
locate_train_df = pd.read_csv(data_root + 'locate_train_preprocessed.csv')
locate_test_df = pd.read_csv(data_root + 'locate_test_preprocessed.csv')

In [460]:
display(locate_train_df.tail(2))
display(locate_test_df.head(2))

print(locate_train_df.shape)
print(locate_test_df.shape)

,주소,target,아파트명,번지,500m이내 버스정류장 수,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적,is_test
568329,서울특별시 구로구 개봉동 476,34900.0,한마을,476,18.0,476.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,37.495303,0.0,126.862409,0.0,서울특별시 구로구 개봉동,1.25,2.772589,경인로 382,1999.0,11.0,18.0,201606.0,59.57,0
568330,서울특별시 구로구 개봉동 476,37500.0,한마을,476,18.0,476.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,37.495303,0.0,126.862409,0.0,서울특별시 구로구 개봉동,1.25,3.401197,경인로 382,1999.0,25.0,18.0,201606.0,84.57,0


,주소,target,아파트명,번지,500m이내 버스정류장 수,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적
0,NaN,NaN,개포6차우성,658-1,12.0,658.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,37.476151,0.0,127.057213,1.0,서울특별시 강남구 개포동,3.5,2.302585,언주로 3,1987.0,5.0,26.0,202307.0,79.9700
1,NaN,NaN,개포더샵트리에,651-1,15.0,651.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,37.484745,1.0,127.057283,0.0,서울특별시 강남구 개포동,3.5,2.708050,개포로 311,2021.0,10.0,15.0,202308.0,108.2017


(568331, 27)
(9272, 26)


# 데이터 병합
- locate_XXX_df 데이터는 내가 선별한 데이터
    - 팀원들의 모든 데이터 병합 후 6~70% 이상 데이터 `drop`
    - 일부 데이터 `drop`

In [461]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
locate_train_df['is_test'] = 0
locate_test_df['is_test'] = 1
concat = pd.concat([locate_train_df, locate_test_df])     # 하나의 데이터로 만들어줍니다.

# Feature Engineering

## 데이터 전처리
1. 시군구 분리
2. 계약년월 -> 계약년월일(datetime)

In [462]:
# 시, 군, 구 분리
concat['시'] = concat['시군구'].apply(lambda x: x.split()[0])
concat['구'] = concat['시군구'].apply(lambda x: x.split()[1])
concat['동'] = concat['시군구'].apply(lambda x: x.split()[2])
# concat.drop('시군구', axis=1, inplace=True)

In [463]:
concat['계약년월'] = concat['계약년월'].astype(int).astype(str) # 소수점 제거 후 문자열로 변환

concat['계약년'] = concat['계약년월'].str[:4]
concat['계약월'] = concat['계약년월'].str[4:]
concat['계약일'] = concat['계약일'].astype(int).astype(str).str.zfill(2)

# concat['계약년월일']을 수치형으로 변환
concat['계약년월일'] = concat['계약년'] + '-' + concat['계약월'] + '-' + concat['계약일']
concat['계약년월일'] = pd.to_datetime(concat['계약년월일'])
concat['계약년월일'] = concat['계약년월일'].apply(lambda x: x.toordinal())

# 숫자형으로 변환
concat['계약년'] = concat['계약년'].astype(int)
concat['계약월'] = concat['계약월'].astype(int)
concat['계약일'] = concat['계약일'].astype(int)

## 파생변수
1. 강남여부
2. 신축아파트
3. 아파트 수명
4. 탑급아파트

In [464]:
gangnam = ['서초구', '강남구', '용산구', '송파구', '성동구']
concat['강남여부'] = concat['구'].apply(lambda x: 1 if x in gangnam else 0)

In [465]:
concat['신축아파트'] = concat['계약년'].apply(lambda x: 1 if x >= 2012 else 0)

In [466]:
# 연차 변수 생성 (2023-계약년도+1)
concat['아파트수명'] = 2023 - concat['건축년도'].astype(int) + 1

In [467]:
top_apartment_name = [
    '자이', '래미안', '파크', '더퍼스트', '센트럴', '포레', 
    '메트로', '에듀', '시티', '힐스테이트', '아이파크', '트리마제',
    '더샵', '더힐', '그랑자이', '아크로', '푸르지오'
]
concat['탑급아파트'] =concat['아파트명'].apply(lambda x: 1 if x in top_apartment_name else 0)

## 결측치 처리

In [468]:
concat.isnull().sum()

주소                9272
target            9272
아파트명                10
번지                   2
500m이내 버스정류장 수      73
본번                   0
부번                   0
75m이내 초등학교여부         0
성취도                  0
50m이내 중학교여부          0
100m이내 중학교여부         0
75m이내 중학교여부          0
대치목동 여부              0
좌표Y                  0
50m이내 초등학교여부         0
좌표X                  0
100m이내 초등학교여부        0
시군구                  0
기본금리                 0
로얄층                  0
도로명                  0
건축년도                 0
층                    0
계약일                  0
계약년월                 0
전용면적                 0
is_test              0
시                    0
구                    0
동                    0
계약년                  0
계약월                  0
계약년월일                0
강남여부                 0
신축아파트                0
아파트수명                0
탑급아파트                0
dtype: int64

In [469]:
concat.drop(['주소', '500m이내 버스정류장 수'], axis=1, inplace=True)

In [470]:
# 아파트명 결측치 처리
concat['아파트명'] = concat['아파트명'].fillna('아파트명없음')

# 번지 결측치 처리
concat['번지'] = concat['번지'].fillna(0)

# 결측치 확인
concat.isnull().sum()

target           9272
아파트명                0
번지                  0
본번                  0
부번                  0
75m이내 초등학교여부        0
성취도                 0
50m이내 중학교여부         0
100m이내 중학교여부        0
75m이내 중학교여부         0
대치목동 여부             0
좌표Y                 0
50m이내 초등학교여부        0
좌표X                 0
100m이내 초등학교여부       0
시군구                 0
기본금리                0
로얄층                 0
도로명                 0
건축년도                0
층                   0
계약일                 0
계약년월                0
전용면적                0
is_test             0
시                   0
구                   0
동                   0
계약년                 0
계약월                 0
계약년월일               0
강남여부                0
신축아파트               0
아파트수명               0
탑급아파트               0
dtype: int64

In [471]:
# 계약년도 2020년 이전 데이터 제거
concat = concat[concat['계약년'] >= 2020]

# 차원축소
- `groupby`
- 데이터의 특성을 파악하여 차원축소

In [472]:
stats = concat.groupby(['계약년', '구'])['target'].agg(['std', 'mean', 'max'])
concat = concat.join(stats, on=['계약년', '구'], rsuffix='_target')

In [473]:
concat['계약년과구(std)'] = concat['std']
concat['계약년과구(mean)'] = concat['mean']
concat['계약년과구(max)'] = concat['max']
concat.drop(['std', 'mean', 'max'], axis=1, inplace=True)
concat.head(1)

,target,아파트명,번지,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적,is_test,시,구,동,계약년,계약월,계약년월일,강남여부,신축아파트,아파트수명,탑급아파트,계약년과구(std),계약년과구(mean),계약년과구(max)
200689,240000.0,개포더샵트리에,651-1,651.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,37.484745,1.0,127.057283,0.0,서울특별시 강남구 개포동,3.5,2.564949,개포로 311,2021.0,8.0,29,202306,105.8064,0,서울특별시,강남구,개포동,2023,6,738700,1,1,3,0,112608.321949,207993.469722,900000.0


In [474]:
stats = concat.groupby(['계약년', '동'])['target'].agg(['std', 'mean', 'max'])
concat = concat.join(stats, on=['계약년', '동'], rsuffix='_target')

In [475]:
concat['계약년과동(std)'] = concat['std']
concat['계약년과동(mean)'] = concat['mean']
concat['계약년과동(max)'] = concat['max']
concat.drop(['std', 'mean', 'max'], axis=1, inplace=True)
concat.head(1)

,target,아파트명,번지,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적,is_test,시,구,동,계약년,계약월,계약년월일,강남여부,신축아파트,아파트수명,탑급아파트,계약년과구(std),계약년과구(mean),계약년과구(max),계약년과동(std),계약년과동(mean),계약년과동(max)
200689,240000.0,개포더샵트리에,651-1,651.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,37.484745,1.0,127.057283,0.0,서울특별시 강남구 개포동,3.5,2.564949,개포로 311,2021.0,8.0,29,202306,105.8064,0,서울특별시,강남구,개포동,2023,6,738700,1,1,3,0,112608.321949,207993.469722,900000.0,65335.877082,169735.59322,370000.0


In [476]:
concat['계약년과동(std)'] = concat['계약년과동(std)'].fillna(concat['계약년과구(std)'])
concat['계약년과동(mean)'] = concat['계약년과동(mean)'].fillna(concat['계약년과구(mean)'])
concat['계약년과동(max)'] = concat['계약년과동(max)'].fillna(concat['계약년과구(max)'])

In [477]:
# 수치형 변수와 범주형 변수 분류
num_col = []
cat_col = []

for column in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[column]):
        num_col.append(column)
    else:
        cat_col.append(column)

print("수치형 변수:", num_col)
print("범주형 변수:", cat_col)

수치형 변수: ['target', '본번', '부번', '75m이내 초등학교여부', '성취도', '50m이내 중학교여부', '100m이내 중학교여부', '75m이내 중학교여부', '대치목동 여부', '좌표Y', '50m이내 초등학교여부', '좌표X', '100m이내 초등학교여부', '기본금리', '로얄층', '건축년도', '층', '계약일', '전용면적', 'is_test', '계약년', '계약월', '계약년월일', '강남여부', '신축아파트', '아파트수명', '탑급아파트', '계약년과구(std)', '계약년과구(mean)', '계약년과구(max)', '계약년과동(std)', '계약년과동(mean)', '계약년과동(max)']
범주형 변수: ['아파트명', '번지', '시군구', '도로명', '계약년월', '시', '구', '동']


In [478]:
df_train = concat.query('is_test==0')
df_test = concat.query('is_test==1')

df_train.drop(['is_test'], axis=1, inplace=True)
df_test.drop(['is_test'], axis=1, inplace=True)
print(df_train.shape, df_test.shape)

(155929, 40) (9272, 40)


/tmp/ipykernel_701202/4216512207.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['is_test'], axis=1, inplace=True)
/tmp/ipykernel_701202/4216512207.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(['is_test'], axis=1, inplace=True)


In [479]:
gangnam_list = ['서초구', '강남구', '용산구', '송파구', '성동구']

# train
gangnam_train = df_train[df_train['구'].isin(gangnam_list)]
non_gangnam_train = df_train[~df_train['구'].isin(gangnam_list)]

# test
gangnam_test = df_train[df_train['구'].isin(gangnam_list)]
non_gangnam_test = df_train[~df_train['구'].isin(gangnam_list)]

In [480]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np
import pandas as pd

# 데이터 타입 확인 및 변환 함수
def convert_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_numeric(df[col])
            except ValueError:
                pass  # 숫자로 변환할 수 없는 경우 그대로 둠
    return df

# 데이터 타입 변환 적용
gangnam_train = convert_dtypes(gangnam_train)
gangnam_test = convert_dtypes(gangnam_test)
non_gangnam_train = convert_dtypes(non_gangnam_train)
non_gangnam_test = convert_dtypes(non_gangnam_test)

# 범주형 변수 재정의
cat_col = [col for col in gangnam_train.columns if gangnam_train[col].dtype == 'object']

# 강남 지역 라벨 인코딩
gangnam_label_encoders = {}
for col in tqdm(cat_col):
    if col != 'is_test':
        lbl = LabelEncoder()
        
        gangnam_train[col] = gangnam_train[col].astype(str)
        gangnam_test[col] = gangnam_test[col].astype(str)
        
        lbl.fit(gangnam_train[col])
        gangnam_train[col] = lbl.transform(gangnam_train[col])
        gangnam_label_encoders[col] = lbl
        
        for label in np.unique(gangnam_test[col]):
            if label not in lbl.classes_:
                lbl.classes_ = np.append(lbl.classes_, label)
        
        gangnam_test[col] = lbl.transform(gangnam_test[col])

# 비강남 지역 라벨 인코딩
non_gangnam_label_encoders = {}
for col in tqdm(cat_col):
    if col != 'is_test':
        lbl = LabelEncoder()
        
        non_gangnam_train[col] = non_gangnam_train[col].astype(str)
        non_gangnam_test[col] = non_gangnam_test[col].astype(str)
        
        lbl.fit(non_gangnam_train[col])
        non_gangnam_train[col] = lbl.transform(non_gangnam_train[col])
        non_gangnam_label_encoders[col] = lbl
        
        for label in np.unique(non_gangnam_test[col]):
            if label not in lbl.classes_:
                lbl.classes_ = np.append(lbl.classes_, label)
        
        non_gangnam_test[col] = lbl.transform(non_gangnam_test[col])

/tmp/ipykernel_701202/1604859707.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col])
  0%|          | 0/7 [00:00<?, ?it/s]/tmp/ipykernel_701202/1604859707.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gangnam_train[col] = gangnam_train[col].astype(str)
/tmp/ipykernel_701202/1604859707.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [481]:
print(gangnam_train.shape, non_gangnam_train.shape)

(32391, 40) (123538, 40)


## 데이터 스플릿(전용면적)

In [482]:
gangnam_train0 = gangnam_train[gangnam_train['전용면적']<35]
gangnam_train35 = gangnam_train[(gangnam_train['전용면적']>=35)&(gangnam_train['전용면적']<85)]
gangnam_train85 = gangnam_train[(gangnam_train['전용면적']>=85)&(gangnam_train['전용면적']<135)]
gangnam_train135 = gangnam_train[(gangnam_train['전용면적']>=135)&(gangnam_train['전용면적']<200)]
gangnam_train200 = gangnam_train[(gangnam_train['전용면적']>=200)]

non_gangnam_test0 = non_gangnam_test[non_gangnam_test['전용면적']<35]
non_gangnam_test35 = non_gangnam_test[(non_gangnam_test['전용면적']>=35)&(non_gangnam_test['전용면적']<85)]
non_gangnam_test85 = non_gangnam_test[(non_gangnam_test['전용면적']>=85)&(non_gangnam_test['전용면적']<135)]
non_gangnam_test135 = non_gangnam_test[(non_gangnam_test['전용면적']>=135)&(non_gangnam_test['전용면적']<200)]
non_gangnam_test200 = non_gangnam_test[(non_gangnam_test['전용면적']>=200)]

y_train0 = gangnam_train0['target']
X_train0 = gangnam_train0.drop(['target', '계약년월일'], axis=1)

y_train35 = gangnam_train35['target']
X_train35 = gangnam_train35.drop(['target', '계약년월일'], axis=1)

y_train85 = gangnam_train85['target']
X_train85 = gangnam_train85.drop(['target', '계약년월일'], axis=1)

y_train135 = gangnam_train135['target']
X_train135 = gangnam_train135.drop(['target', '계약년월일'], axis=1)

y_train200 = gangnam_train200['target']
X_train200 = gangnam_train200.drop(['target', '계약년월일'], axis=1)

y_train0_non = non_gangnam_test0['target']
X_train0_non = non_gangnam_test0.drop(['target', '계약년월일'], axis=1)

y_train35_non = non_gangnam_test35['target']
X_train35_non = non_gangnam_test35.drop(['target', '계약년월일'], axis=1)

y_train85_non = non_gangnam_test85['target']
X_train85_non = non_gangnam_test85.drop(['target', '계약년월일'], axis=1)

y_train135_non = non_gangnam_test135['target']
X_train135_non = non_gangnam_test135.drop(['target', '계약년월일'], axis=1)

y_train200_non = non_gangnam_test200['target']
X_train200_non = non_gangnam_test200.drop(['target', '계약년월일'], axis=1)

# Split the data into train and validation sets
X_train0, X_val0, y_train0, y_val0 = train_test_split(X_train0, y_train0, test_size=0.2, random_state=2023)
X_train35, X_val35, y_train35, y_val35 = train_test_split(X_train35, y_train35, test_size=0.2, random_state=2023)
X_train85, X_val85, y_train85, y_val85 = train_test_split(X_train85, y_train85, test_size=0.2, random_state=2023)
X_train135, X_val135, y_train135, y_val135 = train_test_split(X_train135, y_train135, test_size=0.2, random_state=2023)
X_train200, X_val200, y_train200, y_val200 = train_test_split(X_train200, y_train200, test_size=0.2, random_state=2023)

X_train0_non, X_val0_non, y_train0_non, y_val0_non = train_test_split(X_train0_non, y_train0_non, test_size=0.2, random_state=2023)
X_train35_non, X_val35_non, y_train35_non, y_val35_non = train_test_split(X_train35_non, y_train35_non, test_size=0.2, random_state=2023)
X_train85_non, X_val85_non, y_train85_non, y_val85_non = train_test_split(X_train85_non, y_train85_non, test_size=0.2, random_state=2023)
X_train135_non, X_val135_non, y_train135_non, y_val135_non = train_test_split(X_train135_non, y_train135_non, test_size=0.2, random_state=2023)
X_train200_non, X_val200_non, y_train200_non, y_val200_non = train_test_split(X_train200_non, y_train200_non, test_size=0.2, random_state=2023)

## CatBoost

In [448]:
cat0_gangnam1 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)
cat0_gangnam0 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)

cat35_gangnam1 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)
cat35_gangnam0 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)

cat85_gangnam1 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)
cat85_gangnam0 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)

cat135_gangnam1 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)
cat135_gangnam0 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)

cat200_gangnam1 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)
cat200_gangnam0 = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=10, loss_function='RMSE', eval_metric='RMSE', random_seed=2023)

In [449]:
cat0_gangnam1.fit(
    X_train0_gangnam1, y_train0_gangnam1, 
    eval_set=(X_val0_gangnam1, y_val0_gangnam1),
    use_best_model=True, early_stopping_rounds=50, 
    verbose_eval=100
)

cat0_gangnam0.fit(
    X_train0_gangnam0, y_train0_gangnam0, 
    eval_set=(X_val0_gangnam0, y_val0_gangnam0),
    use_best_model=True, early_stopping_rounds=50,
    verbose_eval=100
)

NameError: name 'X_train0_gangnam1' is not defined

In [35]:
cat35_gangnam1.fit(
    X_train35_gangnam1, y_train35_gangnam1, 
    eval_set=(X_val35_gangnam1, y_val35_gangnam1),
    use_best_model=True, early_stopping_rounds=50, 
    verbose_eval=100
)

cat35_gangnam0.fit(
    X_train35_gangnam0, y_train35_gangnam0, 
    eval_set=(X_val35_gangnam0, y_val35_gangnam0),
    use_best_model=True, early_stopping_rounds=50,
    verbose_eval=100
)

0:	learn: 58343.8719558	test: 57495.7495001	best: 57495.7495001 (0)	total: 13.1ms	remaining: 2m 11s
100:	learn: 29028.2733893	test: 28912.3500073	best: 28912.3500073 (100)	total: 1.29s	remaining: 2m 6s
200:	learn: 19163.1047801	test: 19466.7421840	best: 19466.7421840 (200)	total: 2.57s	remaining: 2m 5s
300:	learn: 15573.9848183	test: 16126.5524280	best: 16126.5524280 (300)	total: 3.85s	remaining: 2m 4s
400:	learn: 13883.1329478	test: 14600.3025603	best: 14600.3025603 (400)	total: 5.11s	remaining: 2m 2s
500:	learn: 12851.2817736	test: 13700.0124090	best: 13700.0124090 (500)	total: 6.38s	remaining: 2m
600:	learn: 12085.9978820	test: 13037.4130859	best: 13037.4130859 (600)	total: 7.64s	remaining: 1m 59s
700:	learn: 11413.1258309	test: 12467.5757023	best: 12467.5757023 (700)	total: 8.9s	remaining: 1m 58s
800:	learn: 10875.8206342	test: 12017.1518012	best: 12017.1518012 (800)	total: 10.2s	remaining: 1m 56s
900:	learn: 10455.3565820	test: 11671.8351903	best: 11671.8351903 (900)	total: 11.5s	

In [36]:
cat85_gangnam1.fit(
    X_train85_gangnam1, y_train85_gangnam1, 
    eval_set=(X_val85_gangnam1, y_val85_gangnam1),
    use_best_model=True, early_stopping_rounds=50, 
    verbose_eval=100
)

cat85_gangnam0.fit(
    X_train85_gangnam0, y_train85_gangnam0, 
    eval_set=(X_val85_gangnam0, y_val85_gangnam0),
    use_best_model=True, early_stopping_rounds=50,
    verbose_eval=100
)

0:	learn: 82431.1224479	test: 86897.6300963	best: 86897.6300963 (0)	total: 8.71ms	remaining: 1m 27s
100:	learn: 40748.9568988	test: 43990.3825808	best: 43990.3825808 (100)	total: 696ms	remaining: 1m 8s
200:	learn: 25208.5066489	test: 28315.3780155	best: 28315.3780155 (200)	total: 1.36s	remaining: 1m 6s
300:	learn: 19148.6587846	test: 22563.0303279	best: 22563.0303279 (300)	total: 2.02s	remaining: 1m 5s
400:	learn: 16435.8971228	test: 20239.6185772	best: 20239.6185772 (400)	total: 2.71s	remaining: 1m 4s
500:	learn: 14876.9024579	test: 19029.6708128	best: 19029.6708128 (500)	total: 3.37s	remaining: 1m 3s
600:	learn: 13812.2411359	test: 18291.2887199	best: 18291.2887199 (600)	total: 4.04s	remaining: 1m 3s
700:	learn: 12973.8198929	test: 17762.7212927	best: 17762.7212927 (700)	total: 4.69s	remaining: 1m 2s
800:	learn: 12231.8595828	test: 17320.6703793	best: 17320.6703793 (800)	total: 5.33s	remaining: 1m 1s
900:	learn: 11602.9098772	test: 16956.3138247	best: 16956.3138247 (900)	total: 6s	re

In [37]:
cat135_gangnam1.fit(
    X_train135_gangnam1, y_train135_gangnam1, 
    eval_set=(X_val135_gangnam1, y_val135_gangnam1),
    use_best_model=True, early_stopping_rounds=50, 
    verbose_eval=100
)

cat135_gangnam0.fit(
    X_train135_gangnam0, y_train135_gangnam0, 
    eval_set=(X_val135_gangnam0, y_val135_gangnam0),
    use_best_model=True, early_stopping_rounds=50,
    verbose_eval=100
)

0:	learn: 116511.8767948	test: 123692.7093728	best: 123692.7093728 (0)	total: 8.83ms	remaining: 1m 28s
100:	learn: 57687.4211924	test: 64844.1955545	best: 64844.1955545 (100)	total: 593ms	remaining: 58.1s
200:	learn: 34997.0637256	test: 41813.2776423	best: 41813.2776423 (200)	total: 1.19s	remaining: 57.8s
300:	learn: 26195.3427489	test: 32893.4356087	best: 32893.4356087 (300)	total: 1.75s	remaining: 56.3s
400:	learn: 22029.4179024	test: 29036.0548233	best: 29036.0548233 (400)	total: 2.33s	remaining: 55.7s
500:	learn: 19577.0423473	test: 27007.2095588	best: 27007.2095588 (500)	total: 2.91s	remaining: 55.3s
600:	learn: 17903.5684200	test: 25775.6190237	best: 25775.6190237 (600)	total: 3.48s	remaining: 54.4s
700:	learn: 16560.7620002	test: 24920.1885323	best: 24920.1885323 (700)	total: 4.03s	remaining: 53.4s
800:	learn: 15394.1124804	test: 24250.3506798	best: 24250.3506798 (800)	total: 4.57s	remaining: 52.5s
900:	learn: 14376.4209999	test: 23736.2553227	best: 23736.2553227 (900)	total: 5.

In [38]:
cat200_gangnam1.fit(
    X_train200_gangnam1, y_train200_gangnam1, 
    eval_set=(X_val200_gangnam1, y_val200_gangnam1),
    use_best_model=True, early_stopping_rounds=50, 
    verbose_eval=100
)

cat200_gangnam0.fit(
    X_train200_gangnam0, y_train200_gangnam0, 
    eval_set=(X_val200_gangnam0, y_val200_gangnam0),
    use_best_model=True, early_stopping_rounds=50,
    verbose_eval=100
)

0:	learn: 169222.5138077	test: 191128.7352203	best: 191128.7352203 (0)	total: 6.64ms	remaining: 1m 6s
100:	learn: 98655.6672043	test: 135961.0709708	best: 135961.0709708 (100)	total: 427ms	remaining: 41.8s
200:	learn: 62939.6328284	test: 109373.7888447	best: 109373.7888447 (200)	total: 835ms	remaining: 40.7s
300:	learn: 44495.1876238	test: 96887.8851985	best: 96887.8851985 (300)	total: 1.23s	remaining: 39.8s
400:	learn: 34346.9761518	test: 91193.4586614	best: 91193.4586614 (400)	total: 1.63s	remaining: 39s
500:	learn: 28291.9572881	test: 88118.4547830	best: 88118.4547830 (500)	total: 2.02s	remaining: 38.4s
600:	learn: 24060.6354619	test: 85968.9812488	best: 85968.9812488 (600)	total: 2.42s	remaining: 37.9s
700:	learn: 20757.2162985	test: 85055.7749951	best: 85055.7749951 (700)	total: 2.84s	remaining: 37.6s
800:	learn: 18054.8802170	test: 84529.9193771	best: 84529.9193771 (800)	total: 3.25s	remaining: 37.3s
900:	learn: 16066.1801217	test: 84092.1464175	best: 84091.7445276 (899)	total: 3

In [39]:
X_test = df_test.drop(['target'], axis=1)
X_test = X_test.sort_values(by='전용면적')
X_test = X_test.sort_index()
X_test.head(30)

,아파트명,번지,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적,시,구,동,계약년,계약월,계약년월일,강남여부,신축아파트,아파트수명,탑급아파트,계약년과구(std),계약년과구(mean),계약년과구(max),계약년과동(std),계약년과동(mean),계약년과동(max),탑급아파트와전용면적(mean),탑급아파트와전용면적(max),탑급아파트와전용면적(min)
0,292,4114,658.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,37.476151,0.0,127.057213,1.0,0,3.5,2.302585,5063,1987.0,5.0,26,42,79.9700,0,0,8,2023,7,1301,1,1,37,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,113637.500000,220000.000000,53800.000000
1,293,4090,651.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,37.484745,1.0,127.057283,0.0,0,3.5,2.708050,390,2021.0,10.0,15,43,108.2017,0,0,8,2023,8,1321,1,1,3,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,94038.457614,192530.991401,49704.727349
2,297,4091,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,0,3.5,2.995732,388,1984.0,15.0,28,42,161.0000,0,0,8,2023,7,1303,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,296083.333333,365000.000000,268500.000000
3,297,4091,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,0,3.5,2.944439,388,1984.0,14.0,10,43,133.4600,0,0,8,2023,8,1316,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,258500.000000,289000.000000,220000.000000
4,297,4091,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,0,3.5,2.397895,388,1984.0,6.0,18,43,104.4300,0,0,8,2023,8,1324,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,209416.666667,249500.000000,145000.000000
5,302,1474,187.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,37.487630,1.0,127.068507,0.0,0,3.5,2.564949,3606,1983.0,8.0,28,42,74.2500,0,0,8,2023,7,1303,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,107686.585366,273000.000000,24000.000000
6,303,1462,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,0,3.5,2.772589,394,1983.0,11.0,2,42,83.2100,0,0,8,2023,7,1277,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,95388.144330,285000.000000,24500.000000
7,303,1462,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,0,3.5,2.833213,394,1983.0,12.0,14,43,73.0200,0,0,8,2023,8,1320,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,189861.111111,270000.000000,40500.000000
8,303,1462,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,0,3.5,2.639057,394,1983.0,9.0,24,43,53.0600,0,0,8,2023,8,1330,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,168711.111111,200000.000000,77500.000000
9,1503,627,1280.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.481161,0.0,127.064899,1.0,0,3.5,3.433987,4033,2019.0,26.0,3,42,126.9280,0,0,8,2023,7,1278,1,1,5,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,395500.000000,436000.000000,355000.000000


In [42]:
X_test0_gangnam1 = X_test[(X_test['전용면적']<35)&(X_test['강남여부']==1)]
X_test0_gangnam0 = X_test[(X_test['전용면적']<35)&(X_test['강남여부']==0)]

X_test35_gangnam1 = X_test[(X_test['전용면적']>=35)&(X_test['전용면적']<85)&(X_test['강남여부']==1)]
X_test35_gangnam0 = X_test[(X_test['전용면적']>=35)&(X_test['전용면적']<85)&(X_test['강남여부']==0)]

X_test85_gangnam1 = X_test[(X_test['전용면적']>=85)&(X_test['전용면적']<135)&(X_test['강남여부']==1)]
X_test85_gangnam0 = X_test[(X_test['전용면적']>=85)&(X_test['전용면적']<135)&(X_test['강남여부']==0)]

X_test135_gangnam1 = X_test[(X_test['전용면적']>=135)&(X_test['전용면적']<200)&(X_test['강남여부']==1)]
X_test135_gangnam0 = X_test[(X_test['전용면적']>=135)&(X_test['전용면적']<200)&(X_test['강남여부']==0)]

X_test200_gangnam1 = X_test[(X_test['전용면적']>=200)&(X_test['강남여부']==1)]
X_test200_gangnam0 = X_test[(X_test['전용면적']>=200)&(X_test['강남여부']==0)]

In [43]:
real_test_pred0_gangnam1 = cat0_gangnam1.predict(X_test0_gangnam1)
real_test_pred0_gangnam0 = cat0_gangnam0.predict(X_test0_gangnam0)

real_test_pred35_gangnam1 = cat35_gangnam1.predict(X_test35_gangnam1)
real_test_pred35_gangnam0 = cat35_gangnam0.predict(X_test35_gangnam0)

real_test_pred85_gangnam1 = cat85_gangnam1.predict(X_test85_gangnam1)
real_test_pred85_gangnam0 = cat85_gangnam0.predict(X_test85_gangnam0)

real_test_pred135_gangnam1 = cat135_gangnam1.predict(X_test135_gangnam1)
real_test_pred135_gangnam0 = cat135_gangnam0.predict(X_test135_gangnam0)

real_test_pred200_gangnam1 = cat200_gangnam1.predict(X_test200_gangnam1)
real_test_pred200_gangnam0 = cat200_gangnam0.predict(X_test200_gangnam0)

In [44]:
X_test0_gangnam1['target'] = real_test_pred0_gangnam1
X_test0_gangnam0['target'] = real_test_pred0_gangnam0

X_test35_gangnam1['target'] = real_test_pred35_gangnam1
X_test35_gangnam0['target'] = real_test_pred35_gangnam0

X_test85_gangnam1['target'] = real_test_pred85_gangnam1
X_test85_gangnam0['target'] = real_test_pred85_gangnam0

X_test135_gangnam1['target'] = real_test_pred135_gangnam1
X_test135_gangnam0['target'] = real_test_pred135_gangnam0

X_test200_gangnam1['target'] = real_test_pred200_gangnam1
X_test200_gangnam0['target'] = real_test_pred200_gangnam0

/tmp/ipykernel_634245/3789095651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test0_gangnam1['target'] = real_test_pred0_gangnam1
/tmp/ipykernel_634245/3789095651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test0_gangnam0['target'] = real_test_pred0_gangnam0
/tmp/ipykernel_634245/3789095651.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [45]:
concat_test = pd.concat([
    X_test0_gangnam1, X_test0_gangnam0,
    X_test35_gangnam1, X_test35_gangnam0,
    X_test85_gangnam1, X_test85_gangnam0,
    X_test135_gangnam1, X_test135_gangnam0,
    X_test200_gangnam1, X_test200_gangnam0
], axis=0)
concat_test = concat_test.sort_index()
concat_test.head(30)

,아파트명,번지,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적,시,구,동,계약년,계약월,계약년월일,강남여부,신축아파트,아파트수명,탑급아파트,계약년과구(std),계약년과구(mean),계약년과구(max),계약년과동(std),계약년과동(mean),계약년과동(max),탑급아파트와전용면적(mean),탑급아파트와전용면적(max),탑급아파트와전용면적(min),target
0,292,4114,658.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,37.476151,0.0,127.057213,1.0,0,3.5,2.302585,5063,1987.0,5.0,26,42,79.9700,0,0,8,2023,7,1301,1,1,37,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,113637.500000,220000.000000,53800.000000,172862.092369
1,293,4090,651.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,37.484745,1.0,127.057283,0.0,0,3.5,2.708050,390,2021.0,10.0,15,43,108.2017,0,0,8,2023,8,1321,1,1,3,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,94038.457614,192530.991401,49704.727349,202228.871243
2,297,4091,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,0,3.5,2.995732,388,1984.0,15.0,28,42,161.0000,0,0,8,2023,7,1303,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,296083.333333,365000.000000,268500.000000,318123.078933
3,297,4091,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,0,3.5,2.944439,388,1984.0,14.0,10,43,133.4600,0,0,8,2023,8,1316,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,258500.000000,289000.000000,220000.000000,261988.773461
4,297,4091,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,0,3.5,2.397895,388,1984.0,6.0,18,43,104.4300,0,0,8,2023,8,1324,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,209416.666667,249500.000000,145000.000000,215704.639897
5,302,1474,187.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,37.487630,1.0,127.068507,0.0,0,3.5,2.564949,3606,1983.0,8.0,28,42,74.2500,0,0,8,2023,7,1303,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,107686.585366,273000.000000,24000.000000,223983.563852
6,303,1462,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,0,3.5,2.772589,394,1983.0,11.0,2,42,83.2100,0,0,8,2023,7,1277,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,95388.144330,285000.000000,24500.000000,235831.415083
7,303,1462,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,0,3.5,2.833213,394,1983.0,12.0,14,43,73.0200,0,0,8,2023,8,1320,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,189861.111111,270000.000000,40500.000000,221026.500741
8,303,1462,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,0,3.5,2.639057,394,1983.0,9.0,24,43,53.0600,0,0,8,2023,8,1330,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,168711.111111,200000.000000,77500.000000,179361.556183
9,1503,627,1280.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.481161,0.0,127.064899,1.0,0,3.5,3.433987,4033,2019.0,26.0,3,42,126.9280,0,0,8,2023,7,1278,1,1,5,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0,395500.000000,436000.000000,355000.000000,384329.490494


In [46]:
ttt = concat_test.loc[:,'target']
ttt = pd.DataFrame(ttt)

ttt= ttt.reset_index()

preds_df= ttt.drop(columns='index')
preds_df= ttt.drop(columns='index')

preds_df2 = pd.DataFrame(preds_df['target'].astype(int), columns=["target"])
preds_df2

preds_df2.to_csv('cat_(2020)_전용면적&강남여부.csv', index=False)

## LGBM

In [483]:
gbm0_gangnam = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)
gbm0_gangnam_non = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)

gbm35_gangnam = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)
gbm35_gangnam_non = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)

gbm85_gangnam = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)
gbm85_gangnam_non = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)

gbm135_gangnam = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)
gbm135_gangnam_non = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)

gbm200_gangnam = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)
gbm200_gangnam_non = lgb.LGBMRegressor(n_estimators=10000, max_depth=50, num_leaves=600, min_child_samples=50,
                            feature_fraction=0.7,
                            bagging_fraction=0.9, verbosity=-1)

In [484]:
# 35m² 미만 모델 학습
gbm0_gangnam.fit(
    X_train0, y_train0, 
    eval_set=[(X_train0, y_train0), (X_val0, y_val0)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('강남 & 전용면적 35m² 미만 모델 학습 완료')

gbm0_gangnam_non.fit(
    X_train0_non, y_train0_non,
    eval_set=[(X_train0_non, y_train0_non), (X_val0_non, y_val0_non)], 
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('비강남 & 전용면적 35m² 미만 모델 학습 완료')

# 35m² 이상 85m² 미만 모델 학습
gbm35_gangnam.fit(
    X_train35, y_train35, 
    eval_set=[(X_train35, y_train35), (X_val35, y_val35)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('강남 & 전용면적 35m² 이상 85m² 미만 모델 학습 완료')

gbm35_gangnam_non.fit(
    X_train35_non, y_train35_non,
    eval_set=[(X_train35_non, y_train35_non), (X_val35_non, y_val35_non)], 
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('비강남 & 전용면적 35m² 이상 85m² 미만 모델 학습 완료')

# 85m² 이상 135m² 미만 모델 학습
gbm85_gangnam.fit(
    X_train85, y_train85, 
    eval_set=[(X_train85, y_train85), (X_val85, y_val85)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('강남 & 전용면적 85m² 이상 135m² 미만 모델 학습 완료')

gbm85_gangnam_non.fit(
    X_train85_non, y_train85_non,
    eval_set=[(X_train85_non, y_train85_non), (X_val85_non, y_val85_non)], 
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('비강남 & 전용면적 85m² 이상 135m² 미만 모델 학습 완료')

# 135m² 이상 200m² 미만 모델 학습
gbm135_gangnam.fit(
    X_train135, y_train135, 
    eval_set=[(X_train135, y_train135), (X_val135, y_val135)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('강남 & 전용면적 135m² 이상 200m² 미만 모델 학습 완료')

gbm135_gangnam_non.fit(
    X_train135_non, y_train135_non,
    eval_set=[(X_train135_non, y_train135_non), (X_val135_non, y_val135_non)], 
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('비강남 & 전용면적 135m² 이상 200m² 미만 모델 학습 완료')

# 200m² 이상 모델 학습
gbm200_gangnam.fit(
    X_train200, y_train200, 
    eval_set=[(X_train200, y_train200), (X_val200, y_val200)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('강남 & 전용면적 200m² 이상 모델 학습 완료')

gbm200_gangnam_non.fit(
    X_train200_non, y_train200_non,
    eval_set=[(X_train200_non, y_train200_non), (X_val200_non, y_val200_non)], 
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=100, show_stdv=True)]
)
print('비강남 & 전용면적 200m² 이상 모델 학습 완료')

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 3631.02	training's l2: 1.31843e+07	valid_1's rmse: 4905.05	valid_1's l2: 2.40595e+07
[200]	training's rmse: 2785.18	training's l2: 7.75721e+06	valid_1's rmse: 4519.03	valid_1's l2: 2.04217e+07
[300]	training's rmse: 2288.85	training's l2: 5.23885e+06	valid_1's rmse: 4430.93	valid_1's l2: 1.96331e+07
[400]	training's rmse: 1963.47	training's l2: 3.85522e+06	valid_1's rmse: 4409.67	valid_1's l2: 1.94452e+07
Early stopping, best iteration is:
[360]	training's rmse: 2082.06	training's l2: 4.33498e+06	valid_1's rmse: 4397.86	valid_1's l2: 1.93411e+07
강남 & 전용면적 35m² 미만 모델 학습 완료
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 1598.72	training's l2: 2.55591e+06	valid_1's rmse: 2102.65	valid_1's l2: 4.42116e+06
[200]	training's rmse: 1201.56	training's l2: 1.44374e+06	valid_1's rmse: 2026.27	valid_1's l2: 4.10579e+06
[300]	training's rmse: 1002.6	training's l2: 1.00521e+06	val

In [485]:
X_test = df_test.drop(['target', '계약년월일'], axis=1)
X_test = X_test.sort_values(by='전용면적')
X_test = X_test.sort_index()
X_test.head(30)

,아파트명,번지,본번,부번,75m이내 초등학교여부,성취도,50m이내 중학교여부,100m이내 중학교여부,75m이내 중학교여부,대치목동 여부,좌표Y,50m이내 초등학교여부,좌표X,100m이내 초등학교여부,시군구,기본금리,로얄층,도로명,건축년도,층,계약일,계약년월,전용면적,시,구,동,계약년,계약월,강남여부,신축아파트,아파트수명,탑급아파트,계약년과구(std),계약년과구(mean),계약년과구(max),계약년과동(std),계약년과동(mean),계약년과동(max)
0,개포6차우성,658-1,658.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,37.476151,0.0,127.057213,1.0,서울특별시 강남구 개포동,3.5,2.302585,언주로 3,1987.0,5.0,26,202307,79.9700,서울특별시,강남구,개포동,2023,7,1,1,37,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
1,개포더샵트리에,651-1,651.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,37.484745,1.0,127.057283,0.0,서울특별시 강남구 개포동,3.5,2.708050,개포로 311,2021.0,10.0,15,202308,108.2017,서울특별시,강남구,개포동,2023,8,1,1,3,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
2,개포우성3차,652,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,서울특별시 강남구 개포동,3.5,2.995732,개포로 307,1984.0,15.0,28,202307,161.0000,서울특별시,강남구,개포동,2023,7,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
3,개포우성3차,652,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,서울특별시 강남구 개포동,3.5,2.944439,개포로 307,1984.0,14.0,10,202308,133.4600,서울특별시,강남구,개포동,2023,8,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
4,개포우성3차,652,652.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,37.483955,1.0,127.056025,0.0,서울특별시 강남구 개포동,3.5,2.397895,개포로 307,1984.0,6.0,18,202308,104.4300,서울특별시,강남구,개포동,2023,8,1,1,40,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
5,개포주공5단지,187,187.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,37.487630,1.0,127.068507,0.0,서울특별시 강남구 개포동,3.5,2.564949,삼성로4길 17,1983.0,8.0,28,202307,74.2500,서울특별시,강남구,개포동,2023,7,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
6,개포주공6단지,185,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,서울특별시 강남구 개포동,3.5,2.772589,개포로 516,1983.0,11.0,2,202307,83.2100,서울특별시,강남구,개포동,2023,7,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
7,개포주공6단지,185,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,서울특별시 강남구 개포동,3.5,2.833213,개포로 516,1983.0,12.0,14,202308,73.0200,서울특별시,강남구,개포동,2023,8,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
8,개포주공6단지,185,185.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.489075,1.0,127.071512,0.0,서울특별시 강남구 개포동,3.5,2.639057,개포로 516,1983.0,9.0,24,202308,53.0600,서울특별시,강남구,개포동,2023,8,1,1,41,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0
9,래미안블레스티지,1280,1280.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.481161,0.0,127.064899,1.0,서울특별시 강남구 개포동,3.5,3.433987,선릉로 8,2019.0,26.0,3,202307,126.9280,서울특별시,강남구,개포동,2023,7,1,1,5,0,112608.321949,207993.469722,900000.0,65335.877082,169735.593220,370000.0


In [486]:
X_test0 = X_test[X_test['전용면적'] < 35]
X_test35 = X_test[(X_test['전용면적'] >= 35) & (X_test['전용면적'] < 85)]
X_test85 = X_test[(X_test['전용면적'] >= 85) & (X_test['전용면적'] < 135)]
X_test135 = X_test[(X_test['전용면적'] >= 135) & (X_test['전용면적'] < 200)]
X_test200 = X_test[X_test['전용면적'] >= 200]

X_test0_gangnam = X_test0[X_test0['강남여부'] == 1]
X_test0_non_gangnam = X_test0[X_test0['강남여부'] == 0]

X_test35_gangnam = X_test35[X_test35['강남여부'] == 1]
X_test35_non_gangnam = X_test35[X_test35['강남여부'] == 0]

X_test85_gangnam = X_test85[X_test85['강남여부'] == 1]
X_test85_non_gangnam = X_test85[X_test85['강남여부'] == 0]

X_test135_gangnam = X_test135[X_test135['강남여부'] == 1]
X_test135_non_gangnam = X_test135[X_test135['강남여부'] == 0]

X_test200_gangnam = X_test200[X_test200['강남여부'] == 1]
X_test200_non_gangnam = X_test200[X_test200['강남여부'] == 0]

In [487]:
def preprocess_data(df):
    # 라벨 인코딩을 위한 딕셔너리
    label_encoders = {}
    
    for col in df.columns:
        if df[col].dtype == 'object':
            if col not in label_encoders:
                label_encoders[col] = LabelEncoder()
            df[col] = label_encoders[col].fit_transform(df[col].astype(str))
        elif col == '계약년월':
            # '계약년월'을 숫자로 변환 (예: '202301' -> 202301)
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

In [488]:
# 모든 데이터프레임에 전처리 적용
X_test = preprocess_data(X_test)

X_test0 = preprocess_data(X_test[X_test['전용면적'] < 35])
X_test35 = preprocess_data(X_test[(X_test['전용면적'] >= 35) & (X_test['전용면적'] < 85)])
X_test85 = preprocess_data(X_test[(X_test['전용면적'] >= 85) & (X_test['전용면적'] < 135)])
X_test135 = preprocess_data(X_test[(X_test['전용면적'] >= 135) & (X_test['전용면적'] < 200)])
X_test200 = preprocess_data(X_test[X_test['전용면적'] >= 200])

# 강남/비강남 분리
X_test0_gangnam = X_test0[X_test0['강남여부'] == 1]
X_test0_non_gangnam = X_test0[X_test0['강남여부'] == 0]

X_test35_gangnam = X_test35[X_test35['강남여부'] == 1]
X_test35_non_gangnam = X_test35[X_test35['강남여부'] == 0]

X_test85_gangnam = X_test85[X_test85['강남여부'] == 1]
X_test85_non_gangnam = X_test85[X_test85['강남여부'] == 0]

X_test135_gangnam = X_test135[X_test135['강남여부'] == 1]
X_test135_non_gangnam = X_test135[X_test135['강남여부'] == 0]

X_test200_gangnam = X_test200[X_test200['강남여부'] == 1]
X_test200_non_gangnam = X_test200[X_test200['강남여부'] == 0]

/tmp/ipykernel_701202/1027443630.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')


In [489]:
# 예측 수행
real_test_pred0_gangnam = gbm0_gangnam.predict(X_test0_gangnam)
real_test_pred0_non_gangnam = gbm0_gangnam_non.predict(X_test0_non_gangnam)

real_test_pred35_gangnam = gbm35_gangnam.predict(X_test35_gangnam)
real_test_pred35_non_gangnam = gbm35_gangnam_non.predict(X_test35_non_gangnam)

real_test_pred85_gangnam = gbm85_gangnam.predict(X_test85_gangnam)
real_test_pred85_non_gangnam = gbm85_gangnam_non.predict(X_test85_non_gangnam)

real_test_pred135_gangnam = gbm135_gangnam.predict(X_test135_gangnam)
real_test_pred135_non_gangnam = gbm135_gangnam_non.predict(X_test135_non_gangnam)

real_test_pred200_gangnam = gbm200_gangnam.predict(X_test200_gangnam)
real_test_pred200_non_gangnam = gbm200_gangnam_non.predict(X_test200_non_gangnam)

In [492]:
# 명시적으로 복사본 생성
X_test0_gangnam = X_test0[X_test0['강남여부'] == 1].copy()
X_test0_non_gangnam = X_test0[X_test0['강남여부'] == 0].copy()

X_test35_gangnam = X_test35[X_test35['강남여부'] == 1].copy()
X_test35_non_gangnam = X_test35[X_test35['강남여부'] == 0].copy()

X_test85_gangnam = X_test85[X_test85['강남여부'] == 1].copy()
X_test85_non_gangnam = X_test85[X_test85['강남여부'] == 0].copy()

X_test135_gangnam = X_test135[X_test135['강남여부'] == 1].copy()
X_test135_non_gangnam = X_test135[X_test135['강남여부'] == 0].copy()

X_test200_gangnam = X_test200[X_test200['강남여부'] == 1].copy()
X_test200_non_gangnam = X_test200[X_test200['강남여부'] == 0].copy()

# .loc[]를 사용하여 값 할당
X_test0_gangnam.loc[:, 'target'] = real_test_pred0_gangnam
X_test0_non_gangnam.loc[:, 'target'] = real_test_pred0_non_gangnam

X_test35_gangnam.loc[:, 'target'] = real_test_pred35_gangnam
X_test35_non_gangnam.loc[:, 'target'] = real_test_pred35_non_gangnam

X_test85_gangnam.loc[:, 'target'] = real_test_pred85_gangnam
X_test85_non_gangnam.loc[:, 'target'] = real_test_pred85_non_gangnam

X_test135_gangnam.loc[:, 'target'] = real_test_pred135_gangnam
X_test135_non_gangnam.loc[:, 'target'] = real_test_pred135_non_gangnam

X_test200_gangnam.loc[:, 'target'] = real_test_pred200_gangnam
X_test200_non_gangnam.loc[:, 'target'] = real_test_pred200_non_gangnam

/tmp/ipykernel_701202/1684917492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test0_gangnam['target'] = real_test_pred0_gangnam
/tmp/ipykernel_701202/1684917492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test0_non_gangnam['target'] = real_test_pred0_non_gangnam
/tmp/ipykernel_701202/1684917492.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [491]:
concat_test = pd.concat([
    X_test0_gangnam, X_test0_gangnam_non,
    X_test35_gangnam, X_test35_gangnam_non,
    X_test85_gangnam, X_test85_gangnam_non,
    X_test135_gangnam, X_test135_gangnam_non,
    X_test200_gangnam, X_test200_gangnam_non
], axis=0)
concat_test = concat_test.sort_index()
concat_test.head(30)

NameError: name 'X_test0_gangnam_non' is not defined

In [52]:
ttt = concat_test.loc[:,'target']
ttt = pd.DataFrame(ttt)

ttt= ttt.reset_index()

preds_df= ttt.drop(columns='index')
preds_df= ttt.drop(columns='index')

preds_df2 = pd.DataFrame(preds_df['target'].astype(int), columns=["target"])
preds_df2

preds_df2.to_csv('lgbm_전용면적&강남여부(0,85,200)_groupby.csv', index=False)

## LGBM optuna

In [38]:
import optuna
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective_lgb(trial, X_train, y_train, X_val, y_val):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'num_leaves': trial.suggest_int('num_leaves', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 1.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 1.0, log=True)
    }

    gbm = lgb.LGBMRegressor(**params, verbosity=-1)

    gbm.fit(
        X_train, y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=50),
                   lgb.log_evaluation(period=100, show_stdv=True)]
    )
    
    val_rmse = gbm.best_score_['valid_1']['rmse']
    return val_rmse

# Define the function to optimize hyperparameters
def optimize_hyperparameters_lgb(X_train, y_train, X_val, y_val, n_trials=100):
    study = optuna.create_study(direction='minimize')
    objective = lambda trial: objective_lgb(trial, X_train, y_train, X_val, y_val)
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_trial.params
    best_model = lgb.LGBMRegressor(**best_params, verbosity=-1)

    best_model.fit(X_train, y_train,
                   eval_set=[(X_train, y_train), (X_val, y_val)],
                   eval_metric='rmse',
                   callbacks=[lgb.early_stopping(stopping_rounds=50),
                              lgb.log_evaluation(period=100, show_stdv=True)])

    return best_model

# 데이터 준비 및 모델 최적화
areas = [0, 35, 85, 135, 200]
gangnam_status = [0, 1]
best_models = {}

for area in areas:
    for gangnam in gangnam_status:
        print(f"Optimizing model for area {area} and gangnam status {gangnam}")
        
        df = globals()[f'df_train{area}_gangnam{gangnam}']
        y = df['target']
        X = df.drop(['target'], axis=1)
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        best_model = optimize_hyperparameters_lgb(X_train, y_train, X_val, y_val, n_trials=20)
        best_models[(area, gangnam)] = best_model


# 테스트 데이터 준비 및 예측
X_test = df_test.drop(['target'], axis=1)
X_test = X_test.sort_values(by='전용면적').sort_index()

for area in areas:
    for gangnam in gangnam_status:
        mask = (X_test['전용면적'] >= area) & (X_test['전용면적'] < (area + 35 if area < 200 else float('inf'))) & (X_test['강남여부'] == gangnam)
        X_test.loc[mask, 'target'] = best_models[(area, gangnam)].predict(X_test.loc[mask])

# 결과 정리
preds_df = pd.DataFrame(X_test['target']).reset_index(drop=True)
preds_df2 = pd.DataFrame(preds_df['target'].astype(int), columns=["target"])

# 결과 저장
preds_df2.to_csv('lgbm_전용면적&강남여부_groupby_optimized.csv', index=False)

[I 2024-07-18 10:08:32,112] A new study created in memory with name: no-name-bd78b04e-39fd-4631-9109-265f29bb5f3a


Optimizing model for area 0 and gangnam status 0
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 1509	training's l2: 2.27709e+06	valid_1's rmse: 2052.32	valid_1's l2: 4.21204e+06
[200]	training's rmse: 1116.77	training's l2: 1.24718e+06	valid_1's rmse: 2040.17	valid_1's l2: 4.16229e+06
Early stopping, best iteration is:
[248]	training's rmse: 1003.77	training's l2: 1.00755e+06	valid_1's rmse: 2033.65	valid_1's l2: 4.13575e+06


[I 2024-07-18 10:08:32,763] Trial 0 finished with value: 2033.6549153105045 and parameters: {'n_estimators': 5600, 'max_depth': 97, 'num_leaves': 565, 'min_child_samples': 40, 'feature_fraction': 0.6330874879129766, 'bagging_fraction': 0.5731577601775177, 'lambda_l1': 4.1702456121171693e-07, 'lambda_l2': 0.047437855012947706}. Best is trial 0 with value: 2033.6549153105045.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2142.23	training's l2: 4.58916e+06	valid_1's rmse: 2285.32	valid_1's l2: 5.22267e+06
[200]	training's rmse: 1599.61	training's l2: 2.55875e+06	valid_1's rmse: 2057.94	valid_1's l2: 4.23512e+06
[300]	training's rmse: 1346.24	training's l2: 1.81235e+06	valid_1's rmse: 2030.69	valid_1's l2: 4.1237e+06


[I 2024-07-18 10:08:33,268] Trial 1 finished with value: 2025.4854113555753 and parameters: {'n_estimators': 7279, 'max_depth': 62, 'num_leaves': 675, 'min_child_samples': 77, 'feature_fraction': 0.5756262715848773, 'bagging_fraction': 0.5364059448978858, 'lambda_l1': 0.0009299294684460513, 'lambda_l2': 6.782394148773459e-08}. Best is trial 1 with value: 2025.4854113555753.


...

[I 2024-07-18 10:13:35,339] Trial 18 finished with value: 60048.99126872274 and parameters: {'n_estimators': 9131, 'max_depth': 22, 'num_leaves': 195, 'min_child_samples': 25, 'feature_fraction': 0.5861282371775616, 'bagging_fraction': 0.5962862758512053, 'lambda_l1': 3.6660654219797623e-06, 'lambda_l2': 9.283895508071495e-05}. Best is trial 11 with value: 58363.24740397685.


[400]	training's rmse: 9299.16	training's l2: 8.64744e+07	valid_1's rmse: 60365.1	valid_1's l2: 3.64394e+09
[500]	training's rmse: 7447.05	training's l2: 5.54586e+07	valid_1's rmse: 60135.9	valid_1's l2: 3.61632e+09
Early stopping, best iteration is:
[523]	training's rmse: 7068.72	training's l2: 4.99668e+07	valid_1's rmse: 60049	valid_1's l2: 3.60588e+09
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 47423.4	training's l2: 2.24898e+09	valid_1's rmse: 82988.1	valid_1's l2: 6.88703e+09
[200]	training's rmse: 36912	training's l2: 1.3625e+09	valid_1's rmse: 74429.2	valid_1's l2: 5.53971e+09
[300]	training's rmse: 31611.6	training's l2: 9.99292e+08	valid_1's rmse: 69977.3	valid_1's l2: 4.89682e+09
[400]	training's rmse: 28214.8	training's l2: 7.96072e+08	valid_1's rmse: 68256.9	valid_1's l2: 4.659e+09
[500]	training's rmse: 25543	training's l2: 6.52446e+08	valid_1's rmse: 67143.8	valid_1's l2: 4.50828e+09
[600]	training's rmse: 23438.7	training's l2: 5.4

[I 2024-07-18 10:13:35,533] Trial 19 finished with value: 64812.318063648854 and parameters: {'n_estimators': 7455, 'max_depth': 38, 'num_leaves': 562, 'min_child_samples': 73, 'feature_fraction': 0.6733083730804044, 'bagging_fraction': 0.5037526784835881, 'lambda_l1': 0.0004214117567843093, 'lambda_l2': 5.887531188130571e-05}. Best is trial 11 with value: 58363.24740397685.


[800]	training's rmse: 20383.5	training's l2: 4.15487e+08	valid_1's rmse: 65242.2	valid_1's l2: 4.25654e+09
[900]	training's rmse: 19082	training's l2: 3.64124e+08	valid_1's rmse: 65006.4	valid_1's l2: 4.22584e+09
[1000]	training's rmse: 17927.7	training's l2: 3.21401e+08	valid_1's rmse: 64876.9	valid_1's l2: 4.20902e+09
Early stopping, best iteration is:
[1005]	training's rmse: 17870	training's l2: 3.19338e+08	valid_1's rmse: 64812.3	valid_1's l2: 4.20064e+09
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 13266.4	training's l2: 1.75997e+08	valid_1's rmse: 60735.2	valid_1's l2: 3.68877e+09
[200]	training's rmse: 7256.8	training's l2: 5.26611e+07	valid_1's rmse: 59075.3	valid_1's l2: 3.48989e+09
[300]	training's rmse: 4984.67	training's l2: 2.4847e+07	valid_1's rmse: 58747.1	valid_1's l2: 3.45122e+09
[400]	training's rmse: 3643.17	training's l2: 1.32727e+07	valid_1's rmse: 58610.8	valid_1's l2: 3.43522e+09
[500]	training's rmse: 2913.49	training's l2

ValueError: Number of features of the model must match the input. Model n_features_ is 42 and input n_features is 43

# Catboost optuna

In [ ]:
import optuna
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

def objective_cat(trial, X_train, y_train, X_val, y_val):
    # Define the hyperparameter search space
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 10000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
    }

    # Create the CatBoostRegressor model with the suggested hyperparameters
    model = CatBoostRegressor(**params, verbose=0)

    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Compute the validation RMSE
    val_rmse = model.get_best_score()['validation']['RMSE']
    return val_rmse

def optimize_hyperparameters_cat(X_train, y_train, X_val, y_val, n_trials=100):
    study = optuna.create_study(direction='minimize')
    objective = lambda trial: objective_cat(trial, X_train, y_train, X_val, y_val)
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_trial.params

    # Create the final model with the best hyperparameters
    best_model = CatBoostRegressor(**best_params, verbose=0)

    # Train the final model with early stopping
    best_model.fit(X_train, y_train,
                   eval_set=(X_val, y_val),
                   early_stopping_rounds=50,
                   verbose=False)

    return best_model

# Prepare data
areas = [0, 35, 85, 135, 200]
X_trains, y_trains, X_vals, y_vals = [], [], [], []

for i in range(len(areas)):
    df = globals()[f'df_train{areas[i]}']
    y = df['target']
    X = df.drop(['target'], axis=1)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2023)
    X_trains.append(X_train)
    y_trains.append(y_train)
    X_vals.append(X_val)
    y_vals.append(y_val)

# Optimize and train models
best_models = []
for i in range(len(areas)):
    print(f"Optimizing model for area {areas[i]}")
    best_model = optimize_hyperparameters_cat(X_trains[i], y_trains[i], X_vals[i], y_vals[i], n_trials=20)
    best_models.append(best_model)

# Prepare test data
X_test = df_test.drop(['target'], axis=1)
X_test = X_test.sort_values(by='전용면적').sort_index()

X_tests = [
    X_test[X_test['전용면적'] < 35],
    X_test[(X_test['전용면적'] >= 35) & (X_test['전용면적'] < 85)],
    X_test[(X_test['전용면적'] >= 85) & (X_test['전용면적'] < 135)],
    X_test[(X_test['전용면적'] >= 135) & (X_test['전용면적'] < 200)],
    X_test[X_test['전용면적'] >= 200]
]

# Make predictions
for i, X_test_subset in enumerate(X_tests):
    X_test_subset['target'] = best_models[i].predict(X_test_subset)

# Combine results
concat_test = pd.concat(X_tests, axis=0).sort_index()
preds_df = pd.DataFrame(concat_test['target']).reset_index(drop=True)
preds_df2 = pd.DataFrame(preds_df['target'].astype(int), columns=["target"])

In [206]:
preds_df2.to_csv('CAT_(2020)_optuna_groupby.csv', index=False)

# XGB optuna

In [278]:
X_train = df_train.drop(['target'], axis=1)
y_train = df_train['target']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [287]:
'''import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split

def objective_xgb(trial, X_train, y_train, X_val, y_val):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
    }

    # Create the XGBRegressor model with the suggested hyperparameters
    model = xgb.XGBRegressor(**params, verbosity=0)

    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        # early_stopping_rounds=50,
        verbose=False
    )
    
    # Compute the validation RMSE
    val_rmse = model.best_score
    return val_rmse

def optimize_hyperparameters_xgb(X_train, y_train, X_val, y_val, n_trials=100):
    study = optuna.create_study(direction='minimize')
    objective = lambda trial: objective_xgb(trial, X_train, y_train, X_val, y_val)
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_trial.params

    # Create the final model with the best hyperparameters
    best_model = xgb.XGBRegressor(**best_params, verbosity=0)

    # Train the final model with early stopping
    best_model.fit(X_train, y_train,
                   eval_set=[(X_val, y_val)],
                #    early_stopping_rounds=50,
                   verbose=False)

    return best_model'''

import optuna
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def objective_xgb(trial, X_train, y_train, X_val, y_val):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
    }

    # Create the XGBRegressor model with the suggested hyperparameters
    model = xgb.XGBRegressor(**params, verbosity=0)

    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # Compute the validation RMSE manually
    y_pred = model.predict(X_val)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return val_rmse

def optimize_hyperparameters_xgb(X_train, y_train, X_val, y_val, n_trials=100):
    study = optuna.create_study(direction='minimize')
    objective = lambda trial: objective_xgb(trial, X_train, y_train, X_val, y_val)
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_trial.params

    # Create the final model with the best hyperparameters
    best_model = xgb.XGBRegressor(**best_params, verbosity=0)

    # Train the final model
    best_model.fit(X_train, y_train,
                   eval_set=[(X_val, y_val)],
                   verbose=False)

    return best_model

# Prepare data
areas = [0, 35, 85, 135, 200]
X_trains, y_trains, X_vals, y_vals = [], [], [], []

for i in range(len(areas)):
    df = globals()[f'df_train{areas[i]}']
    y = df['target']
    X = df.drop(['target'], axis=1)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2023)
    X_trains.append(X_train)
    y_trains.append(y_train)
    X_vals.append(X_val)
    y_vals.append(y_val)

# Optimize and train models
best_models = []
for i in range(len(areas)):
    print(f"Optimizing model for area {areas[i]}")
    best_model = optimize_hyperparameters_xgb(X_trains[i], y_trains[i], X_vals[i], y_vals[i], n_trials=20)
    best_models.append(best_model)

# Prepare test data
X_test = df_test.drop(['target'], axis=1)
X_test = X_test.sort_values(by='전용면적').sort_index()

X_tests = [
    X_test[X_test['전용면적'] < 35],
    X_test[(X_test['전용면적'] >= 35) & (X_test['전용면적'] < 85)],
    X_test[(X_test['전용면적'] >= 85) & (X_test['전용면적'] < 135)],
    X_test[(X_test['전용면적'] >= 135) & (X_test['전용면적'] < 200)],
    X_test[X_test['전용면적'] >= 200]
]

# Make predictions
for i, X_test_subset in enumerate(X_tests):
    X_test_subset['target'] = best_models[i].predict(X_test_subset)

# Combine results
concat_test = pd.concat(X_tests, axis=0).sort_index()
preds_df = pd.DataFrame(concat_test['target']).reset_index(drop=True)
preds_df2 = pd.DataFrame(preds_df['target'].astype(int), columns=["target"])

[I 2024-07-18 01:18:54,375] A new study created in memory with name: no-name-e420e566-385e-4545-9669-8ffa182a6d16


Optimizing model for area 0


[I 2024-07-18 01:19:09,275] Trial 0 finished with value: 2885.8789970326698 and parameters: {'n_estimators': 9369, 'max_depth': 6, 'learning_rate': 0.38414810215505296, 'subsample': 0.9117768832631932, 'colsample_bytree': 0.6152058064341981, 'min_child_weight': 7, 'reg_alpha': 1.03236117746412e-08, 'reg_lambda': 0.000506569873206185}. Best is trial 0 with value: 2885.8789970326698.
[I 2024-07-18 01:19:15,677] Trial 1 finished with value: 2635.065877209678 and parameters: {'n_estimators': 1291, 'max_depth': 10, 'learning_rate': 0.004935462739680397, 'subsample': 0.7258158665353835, 'colsample_bytree': 0.9152714533446977, 'min_child_weight': 3, 'reg_alpha': 0.135023318478852, 'reg_lambda': 0.00013581080293465782}. Best is trial 1 with value: 2635.065877209678.
[I 2024-07-18 01:19:37,774] Trial 2 finished with value: 2818.569311720489 and parameters: {'n_estimators': 8558, 'max_depth': 9, 'learning_rate': 0.009754813544620057, 'subsample': 0.8594106220865421, 'colsample_bytree': 0.9862366

Optimizing model for area 35


[I 2024-07-18 01:23:56,216] Trial 0 finished with value: 6754.469511266919 and parameters: {'n_estimators': 9290, 'max_depth': 6, 'learning_rate': 0.17128038775576923, 'subsample': 0.8792394455774952, 'colsample_bytree': 0.5981240995495948, 'min_child_weight': 9, 'reg_alpha': 0.0005479122322213875, 'reg_lambda': 0.15209210412289356}. Best is trial 0 with value: 6754.469511266919.
[I 2024-07-18 01:24:51,627] Trial 1 finished with value: 8333.532705244443 and parameters: {'n_estimators': 8708, 'max_depth': 5, 'learning_rate': 0.5119010892569192, 'subsample': 0.7537160998636434, 'colsample_bytree': 0.9084515060610099, 'min_child_weight': 1, 'reg_alpha': 0.006399394772556647, 'reg_lambda': 0.00021625906659442158}. Best is trial 0 with value: 6754.469511266919.
[I 2024-07-18 01:25:44,872] Trial 2 finished with value: 7825.580773710882 and parameters: {'n_estimators': 9071, 'max_depth': 4, 'learning_rate': 0.013508952678127127, 'subsample': 0.6741130920660459, 'colsample_bytree': 0.815117659

...

[I 2024-07-18 01:38:54,617] Trial 0 finished with value: 12914.889014604332 and parameters: {'n_estimators': 5543, 'max_depth': 7, 'learning_rate': 0.3342919126123205, 'subsample': 0.6250506499583244, 'colsample_bytree': 0.6873388175696385, 'min_child_weight': 10, 'reg_alpha': 4.4678752660032726e-07, 'reg_lambda': 0.019515422019167807}. Best is trial 0 with value: 12914.889014604332.
[I 2024-07-18 01:39:10,552] Trial 1 finished with value: 13986.357454755649 and parameters: {'n_estimators': 6552, 'max_depth': 7, 'learning_rate': 0.4342783457090585, 'subsample': 0.7565216272851779, 'colsample_bytree': 0.8282858730316287, 'min_child_weight': 5, 'reg_alpha': 4.128806772825028e-06, 'reg_lambda': 0.009776766686446198}. Best is trial 0 with value: 12914.889014604332.
[I 2024-07-18 01:39:20,219] Trial 2 finished with value: 13776.546893126817 and parameters: {'n_estimators': 4088, 'max_depth': 6, 'learning_rate': 0.0013594499344961768, 'subsample': 0.6859750394140403, 'colsample_bytree': 0.69

Optimizing model for area 135


[I 2024-07-18 01:44:14,118] Trial 0 finished with value: 19767.197452632092 and parameters: {'n_estimators': 3958, 'max_depth': 9, 'learning_rate': 0.0031812619379191147, 'subsample': 0.9338551499910813, 'colsample_bytree': 0.7031680067894927, 'min_child_weight': 9, 'reg_alpha': 0.0026599670886774045, 'reg_lambda': 0.007817060389353704}. Best is trial 0 with value: 19767.197452632092.
[I 2024-07-18 01:44:32,414] Trial 1 finished with value: 19593.201765545327 and parameters: {'n_estimators': 9048, 'max_depth': 8, 'learning_rate': 0.024668468359543394, 'subsample': 0.7159171157740511, 'colsample_bytree': 0.998467701201015, 'min_child_weight': 7, 'reg_alpha': 0.0001373105770989337, 'reg_lambda': 6.616402279659594e-08}. Best is trial 1 with value: 19593.201765545327.
[I 2024-07-18 01:44:44,881] Trial 2 finished with value: 19650.72818864185 and parameters: {'n_estimators': 7136, 'max_depth': 7, 'learning_rate': 0.024033090233418524, 'subsample': 0.7636203786571814, 'colsample_bytree': 0.8

Optimizing model for area 200


[I 2024-07-18 01:47:32,469] Trial 0 finished with value: 43534.808250895134 and parameters: {'n_estimators': 3414, 'max_depth': 6, 'learning_rate': 0.0017337311614351466, 'subsample': 0.6203122378828774, 'colsample_bytree': 0.812537940423034, 'min_child_weight': 7, 'reg_alpha': 1.275597686965803e-06, 'reg_lambda': 4.583054100313772e-06}. Best is trial 0 with value: 43534.808250895134.
[I 2024-07-18 01:47:44,285] Trial 1 finished with value: 49480.264382013745 and parameters: {'n_estimators': 7784, 'max_depth': 10, 'learning_rate': 0.089796715677346, 'subsample': 0.85879634170641, 'colsample_bytree': 0.6603629896137853, 'min_child_weight': 3, 'reg_alpha': 0.01436541554780566, 'reg_lambda': 0.013395620323506992}. Best is trial 0 with value: 43534.808250895134.
[I 2024-07-18 01:47:51,506] Trial 2 finished with value: 53068.70473580445 and parameters: {'n_estimators': 9831, 'max_depth': 3, 'learning_rate': 0.6095795351625025, 'subsample': 0.5574009095849792, 'colsample_bytree': 0.532997217

In [288]:
preds_df2.to_csv('XGB_(2020)_optuna_groupby.csv', index=False)